# Protein classification with Subsequence string kernel

This notebook details the utilization of Scikit-Learn to search for the best Support Vector Machine (SVM) model for the classification of protein sequences using the Subsequence string kernel.

## 1. Dataset preparation

This example is about antimicrobial peptides classification. We used the dataset and methodology of the research conducted by P. Bhadra and collaborators, titled "AmPEP: Sequence-based prediction of antimicrobial peptides using distribution patterns of amino acid properties and random forest."  

The data consists of a dataset with a 1:3 positive to negative ratio, AMP/non-AMP peptide sequences. The dataset containing AMP and non-AMP data is freely available at https://sourceforge.net/projects/axpep/files/. 

The original work employs a 10-fold cross-validation Random Forest model and obtains an MCC score of 0.9.

Reference: P. Bhadra, J. Yan, J. Li, S. Fong, and S. W. Siu, "AmPEP: Sequence-based prediction of antimicrobial peptides using distribution patterns of amino acid properties and random forest," Scientific Reports, vol. 8, no. 1, pp. 1–10, 2018.

The first step involves loading the dataset and creating a dataframe.

In [1]:
import pandas as pd

def read_fasta_to_dataframe(fasta_file):
    data = []
    with open(fasta_file, 'r') as file:
        sequence_id = None
        sequence = []
        
        for line in file:
            line = line.strip()
            if line.startswith('>'):
                if sequence_id is not None:
                    data.append([sequence_id, ''.join(sequence)])
                sequence_id = line[1:]
                sequence = []
            else:
                sequence.append(line)
        
        if sequence_id is not None:
            data.append([sequence_id, ''.join(sequence)])
    
    df = pd.DataFrame(data, columns=['seqid', 'sequence'])
    return df

In [2]:
from os import path
amp_seqs_file_path = path.join('data', 'Bhadra-et-al-2018', 'train_AMP_3268.fasta')
amp_df = read_fasta_to_dataframe(amp_seqs_file_path)
amp_df['label'] = 1
amp_df

,seqid,sequence,label
0,AMP_1,AACSDRAHGHICESFKSFCKDSGRNGVKLRANCKKTCGLC,1
1,AMP_2,AAEFPDFYDSEEQMGPHQEAEDEKDRADQRVLTEEEKKELENLAAM...,1
2,AMP_3,AAFFAQQKGLPTQQQNQVSPKAVSMIVNLEGCVRNPYKCPADVWTN...,1
3,AMP_4,AAFRGCWTKNYSPKPCL,1
4,AMP_5,AAGMGFFGAR,1
...,...,...,...
3263,AMP_3264,YIRDFITRRPPFGNI,1
3264,AMP_3265,GILDALTGIL,1
3265,AMP_3266,IFKAIWSGIKRLC,1
3266,AMP_3267,ILGKFCDEIKRIV,1


In [3]:
amp_seqs_file_path = path.join('data', 'Bhadra-et-al-2018', 'train_nonAMP_9777.fasta')
non_amp_df = read_fasta_to_dataframe(amp_seqs_file_path)
non_amp_df['label'] = -1
non_amp_df

,seqid,sequence,label
0,nonamp_1,MNNNTTAPTYTLRGLQLIGWRDMQHALDYLFADGHLKQGTLVAINA...,-1
1,nonamp_2,MKSLLPLAILAALAVAALCYESHESMESYEVSPFTTRRNANTFISP...,-1
2,nonamp_3,MASVTDGKTGIKDASDQNFDYMFKLLIIGNSSVGKTSFLFRYADDT...,-1
3,nonamp_4,MASFQDRAQHTIAQLDKELSKYPVLNNLERQTSVPKVYVILGLVGI...,-1
4,nonamp_5,MRHRSGLRKLNRTSSHRQAMFRNMANSLLRHEVIKTTLPKAKELRR...,-1
...,...,...,...
9772,nonamp_9773,MDNEMTLTFLALSENEALARVAVTGFIAQLDPTIDELSEFKTVVSE...,-1
9773,nonamp_9774,MSKTVVRKNESLDDALRRFKRSVSKAGTLQESRKREFYEKPSVKRK...,-1
9774,nonamp_9775,MRHLVLIGFMGSGKSSLAQELGLALKLEVLDTDMIISERVGLSVRE...,-1
9775,nonamp_9776,MRDLKTYLSVAPVLSTLWFGSLAGLLIEINRFFPDALTFPFFLIRV...,-1


In [4]:
data_df = pd.concat([amp_df, non_amp_df], axis=0)
data_df

,seqid,sequence,label
0,AMP_1,AACSDRAHGHICESFKSFCKDSGRNGVKLRANCKKTCGLC,1
1,AMP_2,AAEFPDFYDSEEQMGPHQEAEDEKDRADQRVLTEEEKKELENLAAM...,1
2,AMP_3,AAFFAQQKGLPTQQQNQVSPKAVSMIVNLEGCVRNPYKCPADVWTN...,1
3,AMP_4,AAFRGCWTKNYSPKPCL,1
4,AMP_5,AAGMGFFGAR,1
...,...,...,...
9772,nonamp_9773,MDNEMTLTFLALSENEALARVAVTGFIAQLDPTIDELSEFKTVVSE...,-1
9773,nonamp_9774,MSKTVVRKNESLDDALRRFKRSVSKAGTLQESRKREFYEKPSVKRK...,-1
9774,nonamp_9775,MRHLVLIGFMGSGKSSLAQELGLALKLEVLDTDMIISERVGLSVRE...,-1
9775,nonamp_9776,MRDLKTYLSVAPVLSTLWFGSLAGLLIEINRFFPDALTFPFFLIRV...,-1


## 2. Scikit-learn grid search integration

Now, we will search for the best value for the hyperparameters *maxlen* and *lambda* of the Subsequence string kernel for this dataset. 

For better performance, we will only use 10% of the samples in hyperparameter selection.

In [6]:
random_seed = 1708  # for reproducing

sampled_amp_df = amp_df.sample(n=len(amp_df) // 10, random_state=random_seed)
sampled_amp_df

,seqid,sequence,label
2969,AMP_2970,RECKTESNTFPGICITKPPCRKACISEKFTDGHCSKILRRCLCTKPC,1
2415,AMP_2416,RMRRSKSGKGSGGSKGSGSKGSKGSKGSGSKGSGSKGGSRPGGGSS...,1
1121,AMP_1122,GLFDIIKKIAESI,1
204,AMP_205,AVDFSSCARMDVPGLSKVAQGLCISSCKFQNCGTGHCEKRGGRPTC...,1
2680,AMP_2681,TLYRRFLCKKMKGRCETACLSFEKKIGTCRADLTPLCCKEKKKH,1
...,...,...,...
700,AMP_701,FLSLIPHAINAVSAIAKHN,1
2692,AMP_2693,TSRCIFYRRKKCS,1
378,AMP_379,EGVRSYLSCWGNRGICLLNRCPGRMRQIGTCLAPRVKCCR,1
1097,AMP_1098,GKIYEQCELAREFKRHGMDGYHGYSLGDWVCTAKHESNFNTAATNY...,1


In [5]:
sampled_non_amp_df = non_amp_df.sample(n=len(non_amp_df) // 10, random_state=random_seed)
sampled_non_amp_df

,seqid,sequence,label
8753,nonamp_8754,MIHKLTSEERKTRLEGLPHWTAVPGRDAIQRSLRFADFNEAFGFMT...,-1
8257,nonamp_8258,MFLNTIKPGEGAKHAKRRVGRGIGSGLGKTAGRGHKGQKSRSGGFH...,-1
5885,nonamp_5886,MYQPDFPPVPFRLGLYPVVDSVQWIERLLDAGVRTLQLRIKDRRDE...,-1
369,nonamp_370,MSFKNPVLGLCQQAAFMLSAARVDQCPADDGLEVAFAGRSNAGKSS...,-1
6825,nonamp_6826,MRVKATLINFKSKLSKSCNRFVSLFRFRVKRPVFIRPLRARHGNVK...,-1
...,...,...,...
9394,nonamp_9395,MSLCELAASGSSLLWPRVLLFGDSITQFSFQQGGWGTLLADRLVRK...,-1
5287,nonamp_5288,MGRFISVSFGLLVVFLSLSGTGADCPSEWSSHEGHCYKVFKLLKTW...,-1
6816,nonamp_6817,MYRMQLLSCIALTLALVANGAPTSSSTGNTMKEVKSLLLDLQLLLE...,-1
7319,nonamp_7320,MDIMKDKIRQALSELDILATEVQIDQWLDYLKLLEKWNKVYNMTAI...,-1


In [7]:
sampled_data_df = pd.concat([sampled_amp_df, sampled_non_amp_df], axis=0)
sampled_data_df

,seqid,sequence,label
2969,AMP_2970,RECKTESNTFPGICITKPPCRKACISEKFTDGHCSKILRRCLCTKPC,1
2415,AMP_2416,RMRRSKSGKGSGGSKGSGSKGSKGSKGSGSKGSGSKGGSRPGGGSS...,1
1121,AMP_1122,GLFDIIKKIAESI,1
204,AMP_205,AVDFSSCARMDVPGLSKVAQGLCISSCKFQNCGTGHCEKRGGRPTC...,1
2680,AMP_2681,TLYRRFLCKKMKGRCETACLSFEKKIGTCRADLTPLCCKEKKKH,1
...,...,...,...
9394,nonamp_9395,MSLCELAASGSSLLWPRVLLFGDSITQFSFQQGGWGTLLADRLVRK...,-1
5287,nonamp_5288,MGRFISVSFGLLVVFLSLSGTGADCPSEWSSHEGHCYKVFKLLKTW...,-1
6816,nonamp_6817,MYRMQLLSCIALTLALVANGAPTSSSTGNTMKEVKSLLLDLQLLLE...,-1
7319,nonamp_7320,MDIMKDKIRQALSELDILATEVQIDQWLDYLKLLEKWNKVYNMTAI...,-1


In [9]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(sampled_data_df['sequence'].values, 
                                                    sampled_data_df['label'].values, 
                                                    stratify=sampled_data_df['label'], 
                                                    random_state=random_seed)
print('Train sequences:',len(X_train))
print('Test sequences:',len(X_test))

Train sequences: 977
Test sequences: 326


In [9]:
# kernel class import
from sys import path as sys_path
sys_path.append('..')
from strkernels import SubsequenceStringKernel

# create a kernel
subsequence_kernel = SubsequenceStringKernel(maxlen=1, ssk_lambda=1)

In [11]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, matthews_corrcoef

# create a support vector classifier with the kernel
clf = SVC(kernel=subsequence_kernel)

# set parameters for grid search
param_grid = {
    'kernel__maxlen': [3, 4, 5],
    'kernel__ssk_lambda': [0.9, 1.0, 1.1],
}

mcc_scorer = make_scorer(matthews_corrcoef)

# create the GridSearchCV object
grid_search = GridSearchCV(estimator=clf, 
                           param_grid=param_grid, 
                           scoring=mcc_scorer, 
                           cv=10,
                           n_jobs=-1, 
                           verbose=3)

# fit the model to the training data
grid_search.fit(X_train, y_train)

# get the best parameters
best_params = grid_search.best_params_

# get the best trained model
best_model = grid_search.best_estimator_

# make predictions using the best model
predictions = best_model.predict(X_test)

# calculate MCC score
MCC_score = matthews_corrcoef(y_test, predictions)

# print the results
print("\nBest parameters:", best_params)
print("MCC score of the best model in test dataset:", MCC_score)

Fitting 10 folds for each of 9 candidates, totalling 90 fits
[CV 1/10] END kernel__maxlen=3, kernel__ssk_lambda=0.9;, score=0.890 total time=13.3min
[CV 8/10] END kernel__maxlen=3, kernel__ssk_lambda=0.9;, score=0.801 total time=13.3min
[CV 2/10] END kernel__maxlen=3, kernel__ssk_lambda=0.9;, score=0.831 total time=13.4min
[CV 7/10] END kernel__maxlen=3, kernel__ssk_lambda=0.9;, score=0.836 total time=13.4min
[CV 4/10] END kernel__maxlen=3, kernel__ssk_lambda=0.9;, score=0.893 total time=13.4min
[CV 6/10] END kernel__maxlen=3, kernel__ssk_lambda=0.9;, score=0.851 total time=13.4min
[CV 5/10] END kernel__maxlen=3, kernel__ssk_lambda=0.9;, score=0.835 total time=13.5min
[CV 3/10] END kernel__maxlen=3, kernel__ssk_lambda=0.9;, score=0.870 total time=13.5min
[CV 9/10] END kernel__maxlen=3, kernel__ssk_lambda=0.9;, score=0.801 total time=12.9min
[CV 2/10] END kernel__maxlen=3, kernel__ssk_lambda=1.0;, score=0.831 total time=12.9min
[CV 3/10] END kernel__maxlen=3, kernel__ssk_lambda=1.0;, sc

### 3. Best model on full dataset

In [10]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(data_df['sequence'].values, 
                                                    data_df['label'].values, 
                                                    stratify=data_df['label'], 
                                                    random_state=random_seed)
print('Train sequences:',len(X_train))
print('Test sequences:',len(X_test))

Train sequences: 9783
Test sequences: 3262


In [11]:
# create a kernel
subsequence_kernel = SubsequenceStringKernel(maxlen=5, ssk_lambda=1.1)

# create a support vector classifier with the kernel
from sklearn.svm import SVC
clf = SVC(kernel=subsequence_kernel)

# train the classifier
clf.fit(X_train, y_train)

In [ ]:
# make predictions using the classifier
predictions = clf.predict(X_test)

# calculate MCC score
MCC_score = matthews_corrcoef(y_test, predictions)

# print the results
print("MCC score of the best model in full test dataset:", MCC_score)